<a href="https://colab.research.google.com/github/db030220/Plogging/blob/yolotrain/yolo_training%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Connect COLAB with Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Check your GPU compatibility with CUDA, as you can see COLAB GPU has CUDA 10.0 installed

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


# Load dataset and darknet folder to COLAB working space by unzip file "darknet.zip"

 * Use syntax !unzip + <path_to_darknet.zip_in_your_GDrive> to extract folder darknet. In my case, "darknet.zip" locates in My Drive/Colab Notebooks/yolo
 

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/YOLO/darknet.zip"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: darknet/data/labels/81_6.png  
 extracting: darknet/data/labels/81_7.png  
  inflating: darknet/data/labels/82_0.png  
  inflating: darknet/data/labels/82_1.png  
  inflating: darknet/data/labels/82_2.png  
  inflating: darknet/data/labels/82_3.png  
  inflating: darknet/data/labels/82_4.png  
  inflating: darknet/data/labels/82_5.png  
  inflating: darknet/data/labels/82_6.png  
 extracting: darknet/data/labels/82_7.png  
  inflating: darknet/data/labels/83_0.png  
  inflating: darknet/data/labels/83_1.png  
  inflating: darknet/data/labels/83_2.png  
  inflating: darknet/data/labels/83_3.png  
  inflating: darknet/data/labels/83_4.png  
  inflating: darknet/data/labels/83_5.png  
 extracting: darknet/data/labels/83_6.png  
 extracting: darknet/data/labels/83_7.png  
  inflating: darknet/data/labels/84_0.png  
  inflating: darknet/data/labels/84_1.png  
  inflating: darknet/data/labels/84_2.png  
  inflating: darknet/data/labels/84_3.pn


#Compile darknet directory with below script:

In [ ]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!make


Cloning into 'darknet'...
remote: Enumerating objects: 15521, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 15521 (delta 0), reused 5 (delta 0), pack-reused 15514
Receiving objects: 100% (15521/15521), 14.19 MiB | 24.38 MiB/s, done.
Resolving deltas: 100% (10412/10412), done.
/content/darknet/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/i

In [ ]:
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!export PKG_CONFIG_PATH=/path/to/directory/containing/opencv.pc:$PKG_CONFIG_PATH


In [ ]:
%cd /content/darknet
!make
!chmod +x ./darknet

/content/darknet
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN  -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -c ./src/convolutional_layer.c -o obj/convolutional_layer.o
Package opencv was not found in the pkg-config search path.
Perhaps you should add the directory containing `opencv.pc'
to the PKG_CONFIG_PATH environment variable
No package 'opencv' found
./src/convolutional_layer.c: In function ‘cudnn_convolutional_setup’:
./src/convolutional_layer.c:148:5: warning: implicit declaration of function ‘cudnnGetConvolutionForwardAlgorithm’; did you mean ‘cudnnGetConvolutionForwardAlgorithm_v7’? [-Wimplicit-function-declaration]
  148 |     cudnnGetConvolutionForwardAlgorithm(cudnn_handle(),
      |     ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
      |     cudnnGetConvolutionForwardAlgorithm_v7
./src/convolutional_layer.c:153:13: error: ‘CUDNN_CONVOLUTION_FWD_SPECIFY_WORKSPACE_LIMIT’ unde

#Save weight during training in your Google Drive

This step is important since COLAB environment will be recycle after 12 hours and all files locate in its working space will be deleted. Here we defines a symbolic link to save the weight directly into our backup folder which we created in our GDrive before. In my case, my backup folder directory is My Drive/YOLOv3_weight/backup.

In [ ]:
!rm /content/darknet/backup -r
!ln -s /content/drive/MyDrive/'Colab Notebooks'/YOLO /content/darknet

ln: failed to create symbolic link '/content/darknet/YOLO': File exists


Install dos2unix to convert train.txt, val.txt, yolo.data, yolo.names, yolov3_custom_train.cfg to unix

In [ ]:
!sudo apt install dos2unix

Reading package lists... Done
Building dependency tree       
Reading state information... Done
dos2unix is already the newest version (7.4.0-2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!dos2unix ./data/train.txt
!dos2unix ./data/train.txt
!dos2unix ./data/percent.data
!dos2unix ./data/percent.names
!dos2unix ./cfg/yolov3.cfg

dos2unix: converting file ./data/train.txt to Unix format...
dos2unix: converting file ./data/train.txt to Unix format...
dos2unix: converting file ./data/percent.data to Unix format...
dos2unix: converting file ./data/percent.names to Unix format...
dos2unix: converting file ./cfg/yolov3.cfg to Unix format...


In [ ]:
%cd /content/darknet
!./darknet detector train data/percent.data cfg/yolov3.cfg darknet53.conv.74 -dont_show --headless

/content/darknet
/bin/bash: ./darknet: Is a directory


In [ ]:
!./darknet/darknet detector test data/percent.data cfg/yolov3.cfg backup/yolov3_custom_train_1000.weights data/images/trash5.png -out out

 GPU isn't used 
 OpenCV version: 4.2.0
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     30       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  30 0.599 BF
   1 conv     30       3 x 3/ 2    608 x 608 x  30 ->  304 x 304 x  30 1.497 BF
   2 conv     30       1 x 1/ 1    304 x 304 x  30 ->  304 x 304 x  30 0.166 BF
   3 conv     30       3 x 3/ 1    304 x 304 x  30 ->  304 x 304 x  30 1.497 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 304 x 304 x  30 0.003 BF
   5 conv     30       3 x 3/ 2    304 x 304 x  30 ->  152 x 152 x  30 0.374 BF
   6 conv     30       1 x 1/ 1    152 x 152 x  30 ->  152 x 152 x  30 0.042 BF
   7 conv     30       3 x 3/ 1    152 x 152 x  30 ->  152 x 152 x  30 0.374 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 152 x 152 x  30 0.001 BF
   9 conv     30       1 x 1/ 1    152 x 152 x  30 ->  152 x 152 x  30 0.042 BF
  10 conv     30       3 x 3/ 1    152 x 152 x 